In [1]:
options(device=pdf) # turn off flashy window on each block execution
# options(repr.plot.width=6, repr.plot.height=4)
Sys.setlocale("LC_ALL", "C")
initialWidth = getOption('width')
options(width=80) # for old folks like me who can't see shit and set font large

[1] "LC_CTYPE=C;LC_NUMERIC=C;LC_TIME=C;LC_COLLATE=C;LC_MONETARY=C;LC_MESSAGES=en_US.UTF-8;LC_PAPER=en_US.UTF-8;LC_NAME=C;LC_ADDRESS=C;LC_TELEPHONE=C;LC_MEASUREMENT=en_US.UTF-8;LC_IDENTIFICATION=C"

In [2]:
data = read.csv('data/loans.csv.xz')
summary(data)
str(data)

 credit.policy                 purpose        int.rate       installment    
 Min.   :0.000   all_other         :2331   Min.   :0.0600   Min.   : 15.67  
 1st Qu.:1.000   credit_card       :1262   1st Qu.:0.1039   1st Qu.:163.77  
 Median :1.000   debt_consolidation:3957   Median :0.1221   Median :268.95  
 Mean   :0.805   educational       : 343   Mean   :0.1226   Mean   :319.09  
 3rd Qu.:1.000   home_improvement  : 629   3rd Qu.:0.1407   3rd Qu.:432.76  
 Max.   :1.000   major_purchase    : 437   Max.   :0.2164   Max.   :940.14  
                 small_business    : 619                                    
 log.annual.inc        dti              fico       days.with.cr.line
 Min.   : 7.548   Min.   : 0.000   Min.   :612.0   Min.   :  179    
 1st Qu.:10.558   1st Qu.: 7.213   1st Qu.:682.0   1st Qu.: 2820    
 Median :10.928   Median :12.665   Median :707.0   Median : 4140    
 Mean   :10.932   Mean   :12.607   Mean   :710.8   Mean   : 4562    
 3rd Qu.:11.290   3rd Qu.:17.950   3rd 

'data.frame':	9578 obs. of  14 variables:
 $ credit.policy    : int  1 1 1 1 1 1 1 1 1 1 ...
 $ purpose          : Factor w/ 7 levels "all_other","credit_card",..: 3 2 3 3 2 2 3 1 5 3 ...
 $ int.rate         : num  0.119 0.107 0.136 0.101 0.143 ...
 $ installment      : num  829 228 367 162 103 ...
 $ log.annual.inc   : num  11.4 11.1 10.4 11.4 11.3 ...
 $ dti              : num  19.5 14.3 11.6 8.1 15 ...
 $ fico             : int  737 707 682 712 667 727 667 722 682 707 ...
 $ days.with.cr.line: num  5640 2760 4710 2700 4066 ...
 $ revol.bal        : int  28854 33623 3511 33667 4740 50807 3839 24220 69909 5630 ...
 $ revol.util       : num  52.1 76.7 25.6 73.2 39.5 51 76.8 68.6 51.1 23 ...
 $ inq.last.6mths   : int  0 0 1 1 0 0 0 0 1 1 ...
 $ delinq.2yrs      : int  0 0 0 0 1 0 0 0 0 0 ...
 $ pub.rec          : int  0 0 0 0 0 0 1 0 0 0 ...
 $ not.fully.paid   : int  0 0 0 0 0 0 1 1 0 0 ...


In [3]:
table(data$not.fully.paid)


   0    1 
8045 1533 

In [4]:
1533/(1533+8045)

[1] 0.1600543

In [8]:
nas = subset(data,
       is.na(log.annual.inc) | is.na(days.with.cr.line) | is.na(revol.util) |
       is.na(inq.last.6mths) | is.na(delinq.2yrs) | is.na(pub.rec))
nrow(nas)
table(nas$not.fully.paid)

[1] 62


 0  1 
50 12 

In [9]:
library(mice)
library(caTools)
#set.seed(144)
#vars.for.imputation = setdiff(names(loans), "not.fully.paid")
#imputed = complete(mice(loans[vars.for.imputation]))
#loans[vars.for.imputation] = imputed

loans = read.csv('data/loans_imputed.csv.xz')
set.seed(144)
split = sample.split(data$not.fully.paid, SplitRatio=0.7)
train = subset(data, split==T)
test = subset(data, split==F)

Loading required package: Rcpp
mice 2.25 2015-11-09


In [10]:
model = glm(not.fully.paid ~ ., data=train, family = 'binomial')
summary(model)


Call:
glm(formula = not.fully.paid ~ ., family = "binomial", data = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2176  -0.6190  -0.4946  -0.3617   2.6436  

Coefficients:
                            Estimate Std. Error z value Pr(>|z|)    
(Intercept)                9.052e+00  1.569e+00   5.770 7.93e-09 ***
credit.policy             -3.488e-01  1.020e-01  -3.421 0.000624 ***
purposecredit_card        -6.289e-01  1.352e-01  -4.651 3.30e-06 ***
purposedebt_consolidation -3.249e-01  9.232e-02  -3.519 0.000433 ***
purposeeducational         1.471e-01  1.757e-01   0.837 0.402400    
purposehome_improvement    1.531e-01  1.488e-01   1.029 0.303470    
purposemajor_purchase     -4.667e-01  2.013e-01  -2.318 0.020443 *  
purposesmall_business      4.125e-01  1.422e-01   2.901 0.003724 ** 
int.rate                   2.915e-01  2.103e+00   0.139 0.889759    
installment                1.279e-03  2.101e-04   6.087 1.15e-09 ***
log.annual.inc            -4.174e-01

In [11]:
10 * -9.308e-03

[1] -0.09308

In [12]:
exp(10 * 9.308e-03)

[1] 1.09755

In [13]:
pred.risk = predict(model, newdata = test, type='response')
test$pred.risk = pred.risk
table(test$not.fully.paid, pred.risk > 0.5)

   
    FALSE TRUE
  0  2387   12
  1   455    3

In [15]:
(2387+3)/(2387+12+455+3)
# baseline
nrow(subset(test, not.fully.paid == F)) / nrow(test)

[1] 0.8365418

[1] 0.8398886

In [17]:
library(ROCR)
ROCRpred = prediction(pred.risk, test$not.fully.paid)
as.numeric(performance(ROCRpred, 'auc')@y.values)

[1] 0.6728841

In [18]:
bivariate = glm(not.fully.paid ~ int.rate, data=train, family = 'binomial')
summary(bivariate)


Call:
glm(formula = not.fully.paid ~ int.rate, family = "binomial", 
    data = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.0547  -0.6271  -0.5442  -0.4361   2.2914  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -3.6726     0.1688  -21.76   <2e-16 ***
int.rate     15.9214     1.2702   12.54   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 5896.6  on 6704  degrees of freedom
Residual deviance: 5734.8  on 6703  degrees of freedom
AIC: 5738.8

Number of Fisher Scoring iterations: 4


In [19]:
cor(train$int.rate, train$fico)

[1] -0.711659

In [22]:
pred.bivariate = predict(bivariate, newdata = test, type = 'response')
max(pred.bivariate)
table(test$not.fully.paid, pred.bivariate >= 0.5)
summary(pred.bivariate)

[1] 0.426624

   
    FALSE
  0  2413
  1   460

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
0.06196 0.11550 0.15080 0.15960 0.18930 0.42660 

In [25]:
library(ROCR)
ROCRpred = prediction(pred.bivariate, test$not.fully.paid)
as.numeric(performance(ROCRpred, 'auc')@y.values)

[1] 0.6239081

In [26]:
10 * exp(0.06 * 3)

[1] 11.97217

In [27]:
test$profit = exp(test$int.rate * 3) - 1
test$profit[test$not.fully.paid == 1] = -1

In [30]:
10 * max(test$profit)

[1] 8.894769

In [32]:
highInterest = subset(test, int.rate >= 0.15)
mean(highInterest$profit)
table(highInterest$not.fully.paid)

[1] 0.2251015


  0   1 
327 110 

In [34]:
110/(110+327)

[1] 0.2517162

In [36]:
cutoff = sort(highInterest$pred.risk, decreasing = F)[100]
cutoff

[1] 0.1757498

In [42]:
selectedLoans = subset(highInterest, pred.risk <= cutoff)
nrow(selectedLoans)

[1] 100

In [47]:
summary(selectedLoans$profit)
str(selectedLoans$profit)
#sum(subset(selectedLoans, profit>0)$profit)
sum(selectedLoans$profit)
table(selectedLoans$not.fully.paid)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-1.0000  0.5830  0.5999  0.3303  0.6320  0.8508 

 num [1:100] -1 0.584 0.604 0.61 -1 ...


[1] 33.0288


 0  1 
82 18 